In [1]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

path = '/opt/ml/input/data/'
dat = pd.read_csv(path + 'FE_total_2_elo2.csv')

In [2]:
dat.drop(['day','year','KnowledgeTag', 'last_answerCode7','last_answerCode8','last_answerCode9','last_answerCode10'], axis = 1, inplace = True)

In [3]:
_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

In [4]:
'''
Train-Valid-Test
'''
# valid 제작하는 함수 새로 개편.
# 예전 valid 제작 함수는 1분 걸렸는데 0.6초만에 끗~
_train['train_valid'] = 0
_train.loc[_train.drop_duplicates(subset='userID', keep = 'last').index, 'train_valid'] = -1
_valid = _train[_train['train_valid'] == -1]
_train = _train[_train['train_valid'] == 0]

# 모델에 적용하기 전 기본적인 데이터 전처리 부분
## 라벨링, 필요없는 칼럼 제거
_train_value = _train['answerCode']
_train.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_valid_value = _valid['answerCode']
_valid.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_test.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

# CatBoost에 적용하기 위해선 문자열 데이터로 변환 필요.
# 만약 범주형이 아니고 연속형(정수, 소수) 성질을 살리고 싶은 변수는 여기 drop에 적어두면 됨.(solve_time 같이.)
cat_columns = list(_train.drop(['solve_time','month'],axis=1).columns)

for col in cat_columns:
    _train[col] = _train[col].astype('str')
    _valid[col] = _valid[col].astype('str')
    _test[col] = _test[col].astype('str')

In [5]:
_train

,userID,assessmentItemID,elouser,eloitem,elotag,elotest,elo,solve_time,b_category,test_category,problem_id,category_st_qcut_5,last_answerCode,last_answerCode2,last_answerCode3,last_answerCode4,last_answerCode5,last_answerCode6,month,hour
0,0,A060001001,0.6040651877696669,-3.354789124318595,-2.9881070823700315,-2.915382465200697,0.9812724475371246,0,6,6001,1,"(-0.001, 9.0]",1,1,1,1,1,1,3,0
1,0,A060001002,0.6040651877696669,-3.0196186232456608,-2.2564708818301846,-2.915382465200697,0.9740093440709234,3,6,6001,2,"(-0.001, 9.0]",1,1,1,1,1,1,3,0
2,0,A060001003,0.6040651877696669,-2.311757216220496,-2.2564708818301846,-2.915382465200697,0.948623075351972,8,6,6001,3,"(-0.001, 9.0]",1,1,1,1,1,1,3,0
3,0,A060001004,0.6040651877696669,-3.0830969581864807,-2.2564708818301846,-2.915382465200697,0.9755688587382544,7,6,6001,4,"(-0.001, 9.0]",1,1,1,1,1,1,3,0
4,0,A060001005,0.6040651877696669,-2.637877399765874,-2.2564708818301846,-2.915382465200697,0.9623824990731704,7,6,6001,5,"(-0.001, 9.0]",1,1,1,1,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2518935,7441,A030071003,-0.0991828504075049,-1.37328857615292,-0.4195089307082005,-0.3807450529003466,0.7814447702800892,40,3,3071,3,"(25.0, 56.0]",0,0,1,1,1,1,6,6
2518936,7441,A030071004,-0.0991828504075049,0.0116435588563796,-0.4195089307082005,-0.3807450529003466,0.4723217217412617,34,3,3071,4,"(25.0, 56.0]",1,0,0,1,1,1,6,6
2518937,7441,A030071005,-0.0991828504075049,0.9244087409246032,-0.4195089307082005,-0.3807450529003466,0.2643283927429934,24,3,3071,5,"(9.0, 25.0]",0,1,0,0,1,1,6,6
2518938,7441,A040165001,-0.0991828504075049,-0.4125350779458633,-0.5764963821048158,-0.4442957486962685,0.5777032902462462,6632178,4,4165,1,"(239.0, 25602295.0]",0,0,1,0,0,1,8,1


In [5]:
'''
Train-Test
'''
# 모델에 적용하기 전 기본적인 데이터 전처리 부분
## 라벨링, 필요없는 칼럼 제거
_train_value = _train['answerCode']
_train.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_test.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'


# CatBoost에 적용하기 위해선 문자열 데이터로 변환 필요.
# 만약 범주형이 아니고 연속형(정수, 소수) 성질을 살리고 싶은 변수는 여기 drop에 적어두면 됨.(solve_time 같이.)
cat_columns = list(_train.drop(['solve_time','month'],axis=1).columns)

for col in cat_columns:
    _train[col] = _train[col].astype('str')
    _test[col] = _test[col].astype('str')

In [6]:
#!pip install catboost
# 가상환경에 패키지 설치 필요할듯
from catboost import CatBoostClassifier

# 하이퍼파라미터 수정할 여지가 많으니 추가나 수정하고 싶은 사람 수정해도 괜찮을듯.
# 일단 지난번에 사용했던 파라미터들 주석으로 들고 옴. 참고하면서 공부하면 좋을듯.

params_cat = {
    "task_type" : "GPU",
    "devices" : '0',
    "random_state": 42,
    'learning_rate': 0.01, 
    'bagging_temperature': 0.15696396388661144, 
    'n_estimators': 4000, 
    'max_depth': 15, 
    'random_strength': 13, 
    'l2_leaf_reg': 5.887526043950164e-06, 
    'min_child_samples': 9, 
    'max_bin': 297, 
    'od_type': 'Iter',
    "cat_features" : cat_columns
}

cat_model = CatBoostClassifier(**params_cat)

# 실제 적용한 모델은 learning_rate, iterations(=n_estimators, 아마도?)만 사용함.
#cat_model = CatBoostClassifier(learning_rate=0.01, random_state = 1998, iterations=4000, task_type="GPU", eval_metric = 'AUC',max_depth = 8)
cat_model.fit(_train, _train_value)

0:	learn: 0.6881122	total: 1.44s	remaining: 1h 35m 44s
1:	learn: 0.6830981	total: 2.86s	remaining: 1h 35m 24s
2:	learn: 0.6785553	total: 4.36s	remaining: 1h 36m 44s
3:	learn: 0.6737801	total: 5.84s	remaining: 1h 37m 10s
4:	learn: 0.6691028	total: 7.23s	remaining: 1h 36m 16s
5:	learn: 0.6651189	total: 7.41s	remaining: 1h 22m 15s
6:	learn: 0.6605031	total: 8.84s	remaining: 1h 24m 3s
7:	learn: 0.6560755	total: 10.3s	remaining: 1h 25m 14s
8:	learn: 0.6517143	total: 11.6s	remaining: 1h 26m 3s
9:	learn: 0.6474519	total: 13.1s	remaining: 1h 26m 47s
10:	learn: 0.6432866	total: 14.5s	remaining: 1h 27m 33s
11:	learn: 0.6395166	total: 14.8s	remaining: 1h 21m 53s
12:	learn: 0.6355674	total: 16.2s	remaining: 1h 23m
13:	learn: 0.6318560	total: 17.7s	remaining: 1h 23m 51s
14:	learn: 0.6279454	total: 19.1s	remaining: 1h 24m 45s
15:	learn: 0.6249503	total: 19.3s	remaining: 1h 20m 12s
16:	learn: 0.6216677	total: 19.6s	remaining: 1h 16m 37s
17:	learn: 0.6182717	total: 21.1s	remaining: 1h 17m 42s
18:	lear

In [7]:
# valid score를 확인해봄.
# train 대비 valid score가 떨어짐. 과적합이 일어남. 마찬가지로 규제하는 하이퍼파라미터 필요.
from sklearn.metrics import accuracy_score, roc_auc_score

print('train score')
_train_pred = cat_model.predict_proba(_train)[:,1]
print(roc_auc_score(_train_value, _train_pred)) # auc
print(accuracy_score(_train_value, np.where(_train_pred >= 0.5, 1, 0))) # acc, 정확도

print('valid score')
_valid_pred = cat_model.predict_proba(_valid)[:,1]
print(roc_auc_score(_valid_value, _valid_pred)) # auc
print(accuracy_score(_valid_value, np.where(_valid_pred >= 0.5, 1, 0))) # acc, 정확도

train score
0.9228878777329391
0.8547337043542838
valid score
0.8552577880861536
0.7776135447460361


In [8]:
# submission 제출하기 위한 코드

_test_pred = cat_model.predict_proba(_test)[:,1]
_test['prediction'] = _test_pred
submission = _test['prediction'].reset_index(drop = True).reset_index()
submission.rename(columns = {'index':'id'}, inplace = True)
submission.to_csv('../output/cat1208elo.csv', index = False)